In [4]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [ ]:
from filterpy.discrete_bayes import update
from filterpy.discrete_bayes import predict


hallway = np.array([1, 1, 0, 0, 0, 0, 0, 0, 1, 0])
kernel = (.1, .8, .1)

prior = np.array([.1] * 10)                         # take an initial radar measurement and define a wide prior around it to be the starting satellite position.    
likelihood = lh_hallway(hallway, z=1, z_prob=.75)   # get radar measurement. Define & obtain likelihood of getting this radar measurement given satellite position.
posterior = update(likelihood, prior)               # update prior belief with radar measurement

prior = predict(posterior, offset=1, kernel=kernel) # incorporate motion & laws to get prior belief on satellite position
likelihood = lh_hallway(hallway, z=1, z_prob=.75)   # get radar measurement. Define likelihood of getting this radar measurement given satellite position.
posterior = update(likelihood, prior)               # update belief with radar measurement

prior = predict(posterior, 1, kernel)               # incorporate motion & laws to get prior belief on satellite position
likelihood = lh_hallway(hallway, z=0, z_prob=.75)   # get radar measurement
posterior = update(likelihood, prior)               # update belief with radar measurement 

# lh_hallway is a likelihood function that gives the likelihood of getting measurement z given each position in the hallway.

# Test connection to C

In [ ]:
import sys
print(sys.prefix)
print(sys.path)

c:\Users\biqin\AppData\Local\Programs\Python\Python312


In [2]:
import testFuncs
testFuncs.getSatParams()


([1.0, 0.0, 1.0], [10.0, 5.0, 2.0])

# Kalman filter(1D)
z = measurement

P = prior variance after prediction (or posterior variance after update)

Q = process noise 

R = measurement noise (or likelihood variance)


In [ ]:
def update(prior, measurement):
    x, P = prior        # mean and variance of prior
    z, R = measurement  # mean and variance of measurement
    
    y = z - x        # residual
    K = P / (P + R)  # Kalman gain

    x = x + K*y      # posterior
    P = (1 - K) * P  # posterior variance
    return gaussian(x, P)

def predict(posterior, movement):
    x, P = posterior # mean and variance of posterior
    dx, Q = movement # mean and variance of movement
    x = x + dx
    P = P + Q
    return gaussian(x, P)

# Kalman filter for a tracking a dog

In [ ]:
import math
import numpy as np
from numpy.random import randn

### generate true trajectory and measurements
def compute_dog_data(z_var, process_var, count=1, dt=1.):
    "returns positions (xs) and measurements (zs)"
    x, vel = 0., 1.
    z_std = math.sqrt(z_var) 
    p_std = math.sqrt(process_var)
    xs, zs = [], []
    for _ in range(count):
        v = vel + (randn() * p_std)
        x += v*dt        
        xs.append(x)
        zs.append(x + randn() * z_std)        
    return np.array(xs), np.array(zs)

In [ ]:
### design state transition matrix F
dt = 0.1
F = np.array([[1, dt],
              [0, 1]])
F

array([[1. , 0.1],
       [0. , 1. ]])

Using Saver class to save the state. Slows the process down but is neat.

In [ ]:
from filterpy.common import Saver
kf = pos_vel_filter([0, .1], R=R, P=P, Q=Q, dt=1.) 
s = Saver(kf)
for i in range(1, 6):
    kf.predict()
    kf.update([i])
    s.save()  # save the current state

In [2]:
from filterpy.common import Q_discrete_white_noise
Q_std = 0.04
dt = 1.0 # time step
Q_discrete_white_noise(dim=2, dt=dt, var=Q_std**2)

array([[0.0004, 0.0008],
       [0.0008, 0.0016]])

In [6]:
from filterpy.common import kinematic_kf
kinematic_kf(dim=3, order=2, dt=1.0, order_by_dim=False)

KalmanFilter object
dim_x = 9
dim_z = 3
dim_u = 0
x = [[0. 0. 0. 0. 0. 0. 0. 0. 0.]].T
P = [[1. 0. 0. 0. 0. 0. 0. 0. 0.]
     [0. 1. 0. 0. 0. 0. 0. 0. 0.]
     [0. 0. 1. 0. 0. 0. 0. 0. 0.]
     [0. 0. 0. 1. 0. 0. 0. 0. 0.]
     [0. 0. 0. 0. 1. 0. 0. 0. 0.]
     [0. 0. 0. 0. 0. 1. 0. 0. 0.]
     [0. 0. 0. 0. 0. 0. 1. 0. 0.]
     [0. 0. 0. 0. 0. 0. 0. 1. 0.]
     [0. 0. 0. 0. 0. 0. 0. 0. 1.]]
x_prior = [[0. 0. 0. 0. 0. 0. 0. 0. 0.]].T
P_prior = [[1. 0. 0. 0. 0. 0. 0. 0. 0.]
           [0. 1. 0. 0. 0. 0. 0. 0. 0.]
           [0. 0. 1. 0. 0. 0. 0. 0. 0.]
           [0. 0. 0. 1. 0. 0. 0. 0. 0.]
           [0. 0. 0. 0. 1. 0. 0. 0. 0.]
           [0. 0. 0. 0. 0. 1. 0. 0. 0.]
           [0. 0. 0. 0. 0. 0. 1. 0. 0.]
           [0. 0. 0. 0. 0. 0. 0. 1. 0.]
           [0. 0. 0. 0. 0. 0. 0. 0. 1.]]
x_post = [[0. 0. 0. 0. 0. 0. 0. 0. 0.]].T
P_post = [[1. 0. 0. 0. 0. 0. 0. 0. 0.]
          [0. 1. 0. 0. 0. 0. 0. 0. 0.]
          [0. 0. 1. 0. 0. 0. 0. 0. 0.]
          [0. 0. 0. 1. 0. 0. 0. 0. 0.]
    

In [20]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

dt = 0.5
F = np.diag([1.]*6)
F[0,3]=dt 
F[1,4]=dt 
F[2,5] =dt
print(F)

[[1.  0.  0.  0.5 0.  0. ]
 [0.  1.  0.  0.  0.5 0. ]
 [0.  0.  1.  0.  0.  0.5]
 [0.  0.  0.  1.  0.  0. ]
 [0.  0.  0.  0.  1.  0. ]
 [0.  0.  0.  0.  0.  1. ]]


In [23]:
from filterpy.kalman import JulierSigmaPoints

sigmas = JulierSigmaPoints(n=2, kappa=1)

def fx(x, dt):
    xout = np.empty_like(x)
    xout[0] = x[1] * dt + x[0]
    xout[1] = x[1]
    return xout

def hx(x):
    return x[:1] # return position [x] 

from numpy.random import randn
from filterpy.kalman import UnscentedKalmanFilter
from filterpy.common import Q_discrete_white_noise

ukf = UnscentedKalmanFilter(dim_x=2, dim_z=1, dt=1., hx=hx, fx=fx, points=sigmas)
print(ukf)

ukf.P *= 10
ukf.R *= .5
ukf.Q = Q_discrete_white_noise(2, dt=1., var=0.03)

print("New UKF ======================================")
print(ukf)

UnscentedKalmanFilter object
x = [0. 0.]
P = [[1. 0.]
     [0. 1.]]
x_prior = [0. 0.]
P_prior = [[1. 0.]
           [0. 1.]]
Q = [[1. 0.]
     [0. 1.]]
R = [[1.]]
S = [[0.]]
K = [[0. 0.]].T
y = [0.]
log-likelihood = -708.3964185322641
likelihood = 2.2250738585072014e-308
mahalanobis = 0.0
sigmas_f = [[0. 0.]
            [0. 0.]
            [0. 0.]
            [0. 0.]
            [0. 0.]]
h = [[0. 0. 0. 0. 0.]].T
Wm = [0.33333333 0.16666667 0.16666667 0.16666667 0.16666667]
Wc = [0.33333333 0.16666667 0.16666667 0.16666667 0.16666667]
residual_x = <ufunc 'subtract'>
residual_z = <ufunc 'subtract'>
msqrt = <function cholesky at 0x0000022CFA795800>
hx = <function hx at 0x0000022CFF039800>
fx = <function fx at 0x0000022CFF03B100>
x_mean = None
z_mean = None
New UKF ======================================
UnscentedKalmanFilter object
x = [0. 0.]
P = [[10.  0.]
     [ 0. 10.]]
x_prior = [0. 0.]
P_prior = [[1. 0.]
           [0. 1.]]
Q = [[0.0075 0.015 ]
     [0.015  0.03  ]]
R = [[0.5]]
S = [

In [25]:
Q_discrete_white_noise(3, dt=0.1, var=0.1)

array([[2.5e-06, 5.0e-05, 5.0e-04],
       [5.0e-05, 1.0e-03, 1.0e-02],
       [5.0e-04, 1.0e-02, 1.0e-01]])

In [30]:
import math
math.radians(180)

3.141592653589793